In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from itertools import product
from collections import defaultdict

In [135]:
np.random.seed(42)
random.seed(42)

n_items = 1000    # Maximum of 1000 unique items belonging to Calking, Glue and Paint
n_warehouses = 10    # 10 warehouses in total
avg_items_per_warehouse = 800  # I kept playing with this number untill I got a good totl number of rowns. 
today = datetime.today()
five_years_ago = today - timedelta(days=5*365)   # Calculate 5 years ago from today to use in last received date

def generate_item_id():   # Item numbers are between 3 and 12 degits
    length = random.randint(3, 12)
    return str(random.randint(10**(length-1), 10**length - 1))

item_ids = set()
while len(item_ids) < n_items:         # Respect the maximum constraint
    item_ids.add(generate_item_id())
item_ids = list(item_ids)

letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'       # 10 warehouses named with 2 upper case letters
warehouse_codes = [''.join(pair) for pair in product(letters, repeat=2)]
warehouse_codes = random.sample(warehouse_codes, n_warehouses)

item_base_costs = {item: round(random.uniform(0.50, 15), 2) for item in item_ids}  # Last costs should be between 50 cents and 5$

warehouse_multipliers = {code: random.uniform(0.9, 1.1) for code in warehouse_codes}  # Warehouse cost multipliers (0.9 to 1.1 range for Â±10% variation)
                                                                                # I need this cause some items cost more to get to certain warehouses

superclass_dist = ['Calking']*2 + ['Glue']*3 + ['Paint']*5  # Superclass distribution (Calking 20%, Glue 30%, Paint 50%)

def generate_warehouse_records(warehouse_code, items):
    records = []
    for item in items:
        base_cost = item_base_costs[item]    # Last cost calculations
        warehouse_multiplier = warehouse_multipliers[warehouse_code] 
        last_cost = round(base_cost * warehouse_multiplier, 3)  # 3 decimal places
        last_cost = max(0.50, min(last_cost, 15))  # Respect 0.50-5$ range       
        days_ago = random.randint(1, 5*365)
        received_date = today - timedelta(days=days_ago)     # Last received date must be from yesterday to 5 years ago
        date_str = f"1{received_date.strftime('%y%m%d')}"       # Date format        
        sales = np.nan if random.random() < 0.2 else random.randint(1, 5000)    # An item can have no sales and can have up to 5000 units sold in
                                                                                    # the last 12 months. if no sales the dataset shows an empty cell
                                                                        # Some items have no 12 months sales most of the times because the item number
                                                                        # Was created for a return, rarely it is because the item is still too new
        activity = ('Discontinued' if days_ago > 3*365 and random.random() < 0.5    # Item activity
                   else random.choice(['Active', 'Discontinued']))        
        superclass = random.choice(superclass_dist)           # Calking or Paint or Glue randomly
        location = f"A-000{random.randint(0, 999):03d}"       # Location where item is stored in warehouse. Must be format A-XXX        
        quantity = (random.randint(0, 500) if activity == 'Discontinued' 
                   else random.randint(0, 10000))        
        shelf_life = (np.nan if random.random() < 0.1                 # Employees sometimes forget to input the shelf life when an item is created
                     else random.choice(range(3, 37, 3)))        
        dispose_code = f"D{random.randint(0, 99):02d}"    # Dispose code must be of format D-XX
        
        records.append({'Item': item, 'Warehouse ID': warehouse_code, 'Last received date': date_str, '12 months sales': sales,
                        'Warehouse activity': activity, 'Superclass': superclass, 'Location': location, 'Quantity on hand': quantity,
                        'Last cost': last_cost, 'Shelf Life': shelf_life, 'Dispose code': dispose_code})
    return records

# Generate data in chunks (suggested by google for memory management)
chunk_size = 10000
all_records = []
for warehouse in warehouse_codes:
    n_items_here = min(np.random.poisson(avg_items_per_warehouse), len(item_ids))         # Use poisson distribution to get the items per werehouse
    n_items_here = max(500, min(n_items_here, 1000))  # No warehouse should have less than 200 items or more than 700 items in the 3 superclasses
    warehouse_items = random.sample(item_ids, n_items_here)    
    for i in range(0, len(warehouse_items), chunk_size):
        all_records.extend(generate_warehouse_records(warehouse, warehouse_items[i:i + chunk_size]))

In [137]:
# Create dataframe with results
df = pd.DataFrame(all_records)

In [139]:
# Prepare "Last received date" column for analysis
# Remove the '1' at the beginning of each row
df['Last received date'] = df['Last received date'].str[1:]

# Then convert to datetime
df['Last received date'] = pd.to_datetime(df['Last received date'], format='%y%m%d')

# Create months since recived column
today = datetime.now()
df['Months since received'] = ((today - df['Last received date']).dt.days / 30).round(2)

# Add column that flags expired items if they have a shelf life
df['Expired'] = np.where((df['Shelf Life'].notna()) & (df['Months since received'] > df['Shelf Life']), 'Y', 'N')
df.head()

# Add column that flags items that will expire in the next month if they have a shelf life
df['Expires next month'] = df.apply(
    lambda row: 'Y' if (
        pd.notna(row['Shelf Life']) and
        (row['Shelf Life'] - row['Months since received'] <= 1) and  # <=1 month remaining
        (row['Shelf Life'] - row['Months since received'] > 0)  # Not already expired
    ) else 'N', axis=1)

# Add column for value per line
df['Value'] = df['Last cost']*df['Quantity on hand']

# Make sure that item is string
df['Item'] = df['Item'].astype(str)

df['Expired_Value'] = df['Value'].where(df['Expired'] == 'Y', 0)

In [141]:
# Save as CSV
df.to_csv('perishables_analysis.csv', index=False)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7955 entries, 0 to 7954
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Item                   7955 non-null   object        
 1   Warehouse ID           7955 non-null   object        
 2   Last received date     7955 non-null   datetime64[ns]
 3   12 months sales        6338 non-null   float64       
 4   Warehouse activity     7955 non-null   object        
 5   Superclass             7955 non-null   object        
 6   Location               7955 non-null   object        
 7   Quantity on hand       7955 non-null   int64         
 8   Last cost              7955 non-null   float64       
 9   Shelf Life             7148 non-null   float64       
 10  Dispose code           7955 non-null   object        
 11  Months since received  7955 non-null   float64       
 12  Expired                7955 non-null   object        
 13  Exp

In [145]:
df.head()

,Item,Warehouse ID,Last received date,12 months sales,Warehouse activity,Superclass,Location,Quantity on hand,Last cost,Shelf Life,Dispose code,Months since received,Expired,Expires next month,Value,Expired_Value
0,42534581118,ZL,2022-11-22,4484.0,Discontinued,Paint,A-000718,78,2.271,33.0,D80,30.33,N,N,177.138,0.000
1,31234703205,ZL,2025-04-30,828.0,Discontinued,Paint,A-000620,358,12.089,6.0,D65,0.67,N,N,4327.862,0.000
2,9037616,ZL,2021-05-08,2316.0,Discontinued,Calking,A-000509,101,9.147,15.0,D66,49.10,Y,N,923.847,923.847
3,775,ZL,2024-04-01,3357.0,Discontinued,Glue,A-000040,280,4.136,12.0,D88,13.80,Y,N,1158.080,1158.080
4,9032,ZL,2022-09-08,23.0,Active,Calking,A-000928,1809,15.000,33.0,D63,32.83,N,Y,27135.000,0.000


In [147]:
df.tail()

,Item,Warehouse ID,Last received date,12 months sales,Warehouse activity,Superclass,Location,Quantity on hand,Last cost,Shelf Life,Dispose code,Months since received,Expired,Expires next month,Value,Expired_Value
7950,73481797,QL,2022-07-21,4544.0,Active,Paint,A-000939,3870,12.474,15.0,D23,34.47,Y,N,48274.380,48274.380
7951,585264473864,QL,2024-03-24,1514.0,Discontinued,Calking,A-000932,8,13.579,18.0,D10,14.07,N,N,108.632,0.000
7952,720,QL,2020-11-30,1491.0,Discontinued,Glue,A-000335,273,2.168,21.0,D15,54.40,Y,N,591.864,591.864
7953,983308055200,QL,2022-07-17,NaN,Active,Glue,A-000010,8988,13.292,36.0,D64,34.60,N,N,119468.496,0.000
7954,22684373,QL,2022-10-06,3383.0,Active,Paint,A-000647,5105,13.909,9.0,D08,31.90,Y,N,71005.445,71005.445


In [149]:
df.sample(10)

,Item,Warehouse ID,Last received date,12 months sales,Warehouse activity,Superclass,Location,Quantity on hand,Last cost,Shelf Life,Dispose code,Months since received,Expired,Expires next month,Value,Expired_Value
4441,411120,JD,2023-07-23,438.0,Discontinued,Glue,A-000097,314,13.082,18.0,D27,22.23,Y,N,4107.748,4107.748
2225,22684373,DN,2023-06-02,2322.0,Discontinued,Paint,A-000407,207,12.123,9.0,D76,23.93,Y,N,2509.461,2509.461
1746,88862539,DN,2022-09-15,1124.0,Active,Glue,A-000172,7120,4.158,33.0,D01,32.60,N,Y,29604.960,0.000
4121,44816,JD,2021-10-20,1200.0,Discontinued,Calking,A-000965,405,6.710,18.0,D05,43.60,Y,N,2717.550,2717.550
4254,538912794,JD,2021-04-08,4844.0,Discontinued,Paint,A-000047,338,11.297,15.0,D57,50.10,Y,N,3818.386,3818.386
710,8022,ZL,2024-12-14,970.0,Discontinued,Calking,A-000917,22,11.290,9.0,D54,5.23,N,N,248.380,0.000
7827,732700038683,QL,2022-06-08,959.0,Discontinued,Paint,A-000357,450,2.688,NaN,D65,35.90,N,N,1209.600,0.000
7314,60707243702,QL,2025-04-01,4658.0,Discontinued,Paint,A-000532,480,2.561,12.0,D42,1.63,N,N,1229.280,0.000
3949,37960031207,WP,2022-05-03,NaN,Discontinued,Paint,A-000215,256,13.136,18.0,D16,37.10,Y,N,3362.816,3362.816
4275,20160926491,JD,2023-11-28,2927.0,Discontinued,Calking,A-000084,334,9.347,12.0,D29,17.97,Y,N,3121.898,3121.898


In [151]:
# Warehouse with the most items
df2=df.groupby(['Warehouse ID'])['Item'].count().reset_index()
df2.max()

Warehouse ID     ZR
Item            831
dtype: object

In [153]:
# Warehouse with the least items
df3=df.groupby(['Warehouse ID'])['Item'].count().reset_index()
df3.min()

Warehouse ID     BM
Item            767
dtype: object

In [155]:
# There are null rows in "12 months sales" and "Shelf Life", check how many
column_nan_count = df.isnull().sum()
column_nan_count

Item                        0
Warehouse ID                0
Last received date          0
12 months sales          1617
Warehouse activity          0
Superclass                  0
Location                    0
Quantity on hand            0
Last cost                   0
Shelf Life                807
Dispose code                0
Months since received       0
Expired                     0
Expires next month          0
Value                       0
Expired_Value               0
dtype: int64